In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn import under_sampling 
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE
import collections, numpy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataFrame = pd.read_csv("/content/drive/MyDrive/GCN Classification/citation_sentiment_corpus.txt", sep = "	", header = None)
dataFrame.columns = ["Source_PaperID", "Target_PaperID", "Sentiment", "Citation_text"]
dataFrame.Sentiment = dataFrame.Sentiment.replace({"o": 1,"p": 2,"n": 0})

In [6]:
dataFrame.columns = ["Source_PaperID", "Target_PaperID", "Sentiment", "Citation_text"]

In [7]:
! pip install tensorflow_text
! pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 39.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import tensorflow_hub as hub
import tensorflow_text as text

In [9]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/2"
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [10]:
text_test = dataFrame['Citation_text']
text_preprocessed = bert_preprocess_model(text_test)

In [11]:
bert_model = hub.KerasLayer(encoder_url)
bert_results = bert_model(text_preprocessed)
bert_results.keys()

dict_keys(['encoder_outputs', 'sequence_output', 'pooled_output', 'default'])

In [12]:
pooled_output = bert_results['pooled_output']
print("Shape of pooled_output:", pooled_output.shape)
print("Values of pooled_output:", pooled_output.numpy())

Shape of pooled_output: (8736, 128)
Values of pooled_output: [[-0.0356641   0.7478295  -0.09831861 ...  0.9589729   0.09576293
   0.97497416]
 [ 0.05013335  0.34173602  0.40624446 ...  0.9043314   0.4538987
   0.98652023]
 [ 0.04367996  0.9301638   0.08268922 ...  0.9410678   0.773179
   0.9911653 ]
 ...
 [ 0.01652739  0.94098914  0.3956203  ...  0.9051377   0.872703
   0.99073964]
 [-0.01301272  0.89886886  0.42208794 ...  0.94257677  0.9150391
   0.99341553]
 [-0.01594586  0.94087964  0.13501185 ...  0.7382965   0.7561564
   0.9807525 ]]


In [13]:
Y = dataFrame['Sentiment']
print(Y)

0       1
1       1
2       1
3       1
4       1
       ..
8731    2
8732    1
8733    1
8734    2
8735    1
Name: Sentiment, Length: 8736, dtype: int64


In [31]:
X = pooled_output

In [17]:
Y

0       1
1       1
2       1
3       1
4       1
       ..
8731    2
8732    1
8733    1
8734    2
8735    1
Name: Sentiment, Length: 8736, dtype: int64

In [32]:

X = np.array(X)

In [25]:
X.shape

(8736, 128)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [28]:
X_train.shape

(16119, 128)

In [34]:
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
acc_lr = round(accuracy_score(y_test, y_pred_lr) * 100, 2)
pre_score_lr = round(precision_score(y_test, y_pred_lr, average='weighted', zero_division=1) * 100, 2)
recall_score_lr = round(recall_score(y_test, y_pred_lr, average='weighted') * 100, 2)
fl_score_lr = round(f1_score(y_test, y_pred_lr, average='weighted') * 100, 2)
print(acc_lr)
print(pre_score_lr)
print(recall_score_lr)
print(fl_score_lr)

86.19
80.95
86.19
80.28


In [35]:
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)
y_pred_dt = DTclf.predict(X_test) 
acc_dt = round(accuracy_score(y_test, y_pred_dt) * 100,2)
pre_score_dt = round(precision_score(y_test, y_pred_dt, average='weighted', zero_division=1) * 100,2)
recall_score_dt = round(recall_score(y_test, y_pred_dt, average='weighted') * 100,2)
fl_score_dt = round(f1_score(y_test, y_pred_dt, average='weighted') * 100,2)
print(acc_dt)
print(pre_score_dt)
print(recall_score_dt)
print(fl_score_dt)

77.6
77.08
77.6
77.34


In [36]:
RFclf = RandomForestClassifier()
RFclf.fit(X_train, y_train)
y_pred_RF = RFclf.predict(X_test)
acc_rf = round(accuracy_score(y_test, y_pred_RF) * 100,2)
pre_score_rf = round(precision_score(y_test, y_pred_RF, average='weighted', zero_division=1) * 100,2)
recall_score_rf = round(recall_score(y_test, y_pred_RF, average='weighted') * 100,2)
fl_score_rf = round(f1_score(y_test, y_pred_RF, average='weighted') * 100,2)
print(acc_rf)
print(pre_score_rf)
print(recall_score_rf)
print(fl_score_rf)

86.15
81.42
86.15
80.54


In [37]:
SVMclf = svm.SVC()
SVMclf.fit(X_train, y_train)
y_pred_svm = SVMclf.predict(X_test)
acc_svm = round(accuracy_score(y_test, y_pred_svm) * 100,2)
pre_score_svm = round(precision_score(y_test, y_pred_svm, average='weighted', zero_division=1) * 100,2)
recall_score_svm = round(recall_score(y_test, y_pred_svm, average='weighted') * 100,2)
fl_score_svm = round(f1_score(y_test, y_pred_svm, average='weighted') * 100,2)
print(acc_svm)
print(pre_score_svm)
print(recall_score_svm)
print(fl_score_svm)

86.0
87.96
86.0
79.52


In [41]:
2*87.96*86

15129.119999999999

In [42]:
87.96+86

173.95999999999998

In [43]:
15129.119/173.959

86.9694525721578

In [38]:
ADAclf = AdaBoostClassifier(n_estimators=100, random_state=0)
ADAclf.fit(X_train, y_train)
y_pred_ada = ADAclf.predict(X_test)
acc_ada = round(accuracy_score(y_test, y_pred_ada) * 100,2)
pre_score_ada = round(precision_score(y_test, y_pred_ada, average='weighted', zero_division=1) * 100,2)
recall_score_ada = round(recall_score(y_test, y_pred_ada, average='weighted') * 100,2)
fl_score_ada = round(f1_score(y_test, y_pred_ada, average='weighted') * 100,2)
print(acc_ada)
print(pre_score_ada)
print(recall_score_ada)
print(fl_score_ada)

84.89
77.83
84.89
80.17


In [39]:
ETclf = ExtraTreesClassifier(n_estimators=100, random_state=0)
ETclf.fit(X_train, y_train)
y_pred_et = ETclf.predict(X_test)
acc_et = round(accuracy_score(y_test, y_pred_et) * 100,2)
pre_score_et = round(precision_score(y_test, y_pred_et, average='weighted', zero_division=1) * 100,2)
recall_score_et = round(recall_score(y_test, y_pred_et, average='weighted') * 100,2)
fl_score_et = round(f1_score(y_test, y_pred_et, average='weighted') * 100,2)
print(acc_et)
print(pre_score_et)
print(recall_score_et)
print(fl_score_et)

86.07
80.59
86.07
80.45


In [40]:
SGDclf = SGDClassifier()
SGDclf.fit(X_train, y_train)
y_pred_sgd = SGDclf.predict(X_test)
acc_sgd = round(accuracy_score(y_test, y_pred_sgd) * 100,2)
pre_score_sgd = round(precision_score(y_test, y_pred_sgd, average='weighted', zero_division=1) * 100,2)
recall_score_sgd = round(recall_score(y_test, y_pred_sgd, average='weighted') * 100,2)
fl_score_sgd = round(f1_score(y_test, y_pred_sgd, average='weighted') * 100,2)
print(acc_sgd)
print(pre_score_sgd)
print(recall_score_sgd)
print(fl_score_sgd)

83.48
79.35
83.48
80.65
